In [1]:
import psycopg2

## Create PUF URL Tracking Database

This is a database with a small number of tables used to track the PUF URLs and their JSON child URLs.

    psql
    # CREATE USER ascaproject WITH PASSWORD 'password'
    # CREATE DATABASE acaproject
    # GRANT ALL PRIVILEGES ON DATABASE acaproject TO acaproject
    # GRANT ALL PRIVILEGES ON DATABASE acaproject TO rcordell (or whoever the main id is)
    # \q
    
   

In [2]:
def create_tables(cursor):
    cursor.execute("CREATE TABLE URLS ("
        "  id SERIAL PRIMARY KEY,"
        "  url TEXT NOT NULL "
        ")")

    cursor.execute (
        "CREATE TABLE URLTYPES ("
        "  id SERIAL PRIMARY KEY,"
        "  type TEXT NOT NULL"
        ")")

    cursor.execute (
        "CREATE TABLE RETRIEVAL_STATUS ("
        "  id SERIAL PRIMARY KEY,"
        "  status TEXT NOT NULL"
        ")")

    cursor.execute (
        "CREATE TABLE JSONURLS ("
        "  id SERIAL PRIMARY KEY,"
        "  url TEXT NOT NULL,"
        "  parent_url INT references URLS(id),"
        "  type int references URLTYPES(id),"
        "  status INT references RETRIEVAL_STATUS(id),"
        "  message TEXT, "
        "  s3key TEXT "
        ")")

In [3]:
def create_statics(cursor):
    cursor.execute("INSERT INTO URLTYPES (type) VALUES ('PROVIDER'),('PLAN'),('FORMULARY')")
    cursor.execute("INSERT INTO RETRIEVAL_STATUS (status) VALUES ('NEW'),('PROCESSED'),('ERROR')")

In [4]:
DB_NAME = 'acaproject'

conn=psycopg2.connect(user="acaproject", 
                      database="acaproject", 
                      password="test1234", 
                      host="w210.cxihwctrc5di.us-west-1.rds.amazonaws.com",
                      port="5432")
cur = conn.cursor()

In [5]:
create_tables(cur)
create_statics(cur)
conn.commit()

In [8]:
# Test an insert into the tables

# First, insert a PUF URL
cur.execute("INSERT INTO URLS (url) VALUES ('http://this.com'),('http://that.com')")

In [9]:
# Next, insert a row into the jsonurl table that references the mrpuf and status and types tables
cur.execute(
    "INSERT INTO JSONURLS (url, parent_url, type, status) "
    "VALUES ('https://providers.json', "
    "(SELECT ID FROM URLS WHERE url='http://this.com'), "
    "(SELECT ID FROM URLTYPES WHERE type='PROVIDER'), "
    "(SELECT ID FROM retrieval_status WHERE status='NEW'))"
)

In [10]:
cur.execute("SELECT * from JSONURLS")

In [11]:
print cur.fetchall()

[(1, 'https://providers.json', 1, 1, 1, None, None)]


In [12]:
cur.close()
conn.commit()
conn.close()

## Process Results Files into DB

I've been running the JSON download into S3 by writing the results to files and pulling those files into git. I'm going to move those results into the new Postgres database

In [1]:
import psycopg2
import json
import csv
from urlparse import urlparse
import requests
from requests.exceptions import SSLError

In [20]:
DB_NAME = 'acaproject'

conn=psycopg2.connect(user="acaproject", 
                      database="acaproject", 
                      password="test1234", 
                      host="w210.cxihwctrc5di.us-west-1.rds.amazonaws.com",
                      port="5432")
cur = conn.cursor()

In [3]:
# reconstitute the dictionary from the file on disk
def load_urls(urlfile):
    urls = []
    with open(urlfile, 'r') as infile:
        for line in infile.readlines():
            urls.append(json.loads(line.strip()))
    return urls

In [4]:
csv_filename = 'machine-readable-url-puf.csv'
provider_urlfile = 'provider-urls.txt'
plan_urlfile = 'plan-urls.txt'
formulary_urlfile = 'formulary-urls.txt'

# load up the URL files into lists
providers = load_urls(provider_urlfile)
plans = load_urls(plan_urlfile)
formularys = load_urls(formulary_urlfile)

# open the CSV file and get all the top level URLs
url_list = []
with open(csv_filename, 'r') as csvfile:
    count = 0
    bad_url = 0
    skipped = 0
    rows = csv.DictReader(csvfile)
    for row in rows:
        _url = row['URL Submitted']
        
        if urlparse(_url).scheme:
            if _url not in url_list:
                url_list.append(_url)
                cur.execute("INSERT INTO urls (url) VALUES (%(u)s)", {'u': _url})

conn.commit()

In [6]:
# Populate the URLS table using only the unique URLS from the CSV file URLs
errors= []
all_urls = []
cur.execute("SELECT DISTINCT url FROM urls")
url_list = cur.fetchall()

for u in url_list:
    try:
        _url = u[0]
        response = requests.get(_url)
        links = json.loads(response.content)
        if 'provider_urls' in links:
            for provider_url in links['provider_urls']:
                if provider_url not in all_urls:
                    cur.execute("INSERT INTO jsonurls (url, parent_url, type, status) "
                                "VALUES (%(a)s, "
                                "(SELECT id FROM urls WHERE url=%(b)s), "
                                "(SELECT id FROM urltypes WHERE type=%(c)s), "
                                "(SELECT id FROM retrieval_status WHERE status=%(d)s)"
                                ")",{'a': provider_url, 
                                     'b': _url, 
                                     'c': 'PROVIDER', 
                                     'd': 'NEW'
                                })
                    all_urls.append(provider_url)
     
        if 'formulary_urls' in links:
            for formulary_url in links['formulary_urls']:
                if formulary_url not in all_urls:
                    cur.execute("INSERT INTO jsonurls (url, parent_url, type, status) "
                                "VALUES (%(a)s, "
                                "(SELECT id FROM urls WHERE url=%(b)s), "
                                "(SELECT id FROM urltypes WHERE type=%(c)s), "
                                "(SELECT id FROM retrieval_status WHERE status=%(d)s)"
                                ")",{'a': formulary_url, 
                                     'b': _url, 
                                     'c': 'FORMULARY', 
                                     'd': 'NEW'
                                })
                    all_urls.append(formulary_url)
                
        if 'plan_urls' in links:
            for plan_url in links['plan_urls']:
                if plan_url not in all_urls:
                    cur.execute("INSERT INTO jsonurls (url, parent_url, type, status) "
                                "VALUES (%(a)s, "
                                "(SELECT id FROM urls WHERE url=%(b)s), "
                                "(SELECT id FROM urltypes WHERE type=%(c)s), "
                                "(SELECT id FROM retrieval_status WHERE status=%(d)s)"
                                ")",{'a': plan_url, 
                                     'b': _url, 
                                     'c': 'PLAN', 
                                     'd': 'NEW'
                                })
                    all_urls.append(plan_url)
                    
    except ValueError as ve:
        print "JSON load failed with this url:"
        print _url
        errors.append({'url': _url, 'error': 'JSON load failed', 'message': str(ve)})
    except SSLError as se:
        print "SSL Error attempting to negotiate:"
        print _url
        errors.append({'url': _url, 'error': 'SSL Error', 'message': str(se)})
    except Exception as inst:
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        errors.append({'url': _url, 'error': str(type(inst)), 'message': str(inst)})
        
print len(all_urls)
conn.commit()

SSL Error attempting to negotiate:
https://esbgatewaypub.medica.com:443/rest/QHP/cms-data-index.json?HIOSID=57845&fmt=json
<class 'requests.exceptions.ConnectionError'>
(ProtocolError('Connection aborted.', BadStatusLine("''",)),)
('Connection aborted.', BadStatusLine("''",))
JSON load failed with this url:
http://www.christushealthplan.org/nm2016healthexchangeformulary
JSON load failed with this url:
http://www.ohcoop.org/wp-content/uploads/99389OR-OHCOOP-data-index.json
JSON load failed with this url:
http://www.dentaquest.com/marketplace/md
SSL Error attempting to negotiate:
https://www.dominionmembers.com/index.json
SSL Error attempting to negotiate:
https://www.christushealthplan.org/workfiles/json/Christusdrugs.json
SSL Error attempting to negotiate:
https://esbgatewaypub.medica.com:443/rest/QHP/cms-data-index.json?HIOSID=20305&fmt=json
SSL Error attempting to negotiate:
https://secure.sierraspectrumppo.com/cmsjson/
SSL Error attempting to negotiate:
https://esbgatewaypub.medica.

In [15]:
# Update the status of the main URL table from the errors list
for item in errors:
    cur.execute("UPDATE urls SET "
                "status=(SELECT id FROM retrieval_status WHERE status='ERROR'),"
                "message=%(m)s "
                "WHERE url=%(u)s",
                {'m': item['message'], 'u': item['url']}
               )
cur.execute("UPDATE urls SET "
            "status=(SELECT id FROM retrieval_status WHERE status='PROCESSED') " 
            "WHERE status is NULL"
           
           )    

conn.commit()

In [31]:
def update_jsonstatus(cursor, line):
    try:
        a = json.loads(line.strip())
        if 's3key' in a:
            cur.execute("UPDATE jsonurls SET "
                        "status=(SELECT id FROM retrieval_status WHERE status='PROCESSED'), "
                        "s3key=%(key)s "
                        "WHERE url=%(u)s",
                        {'key': a['s3key'], 'u': a['url']}
                       )
    except ValueError as ve:
        pass

In [32]:
# update the jsonurls table from the results files used in the initial downloads
cur = conn.cursor()
for fname in ['results-provider-urls.txt', 'results-formulary-urls.txt', 'results-plan-urls.txt']:
    with open(fname,'r') as resfile:
        for line in resfile.readlines():
            update_jsonstatus(cur, line)

In [33]:
conn.commit()

### URL Status

At this point we now have all the URLs in a PostgreSQL database in an AWS RDS instance and their status and S3 bucket/key location. From now on we can use the database to keep track of the data update status. There are still 939 URLs that haven't been downloaded, mostly from the Provider collection. We'll fill in that gap with new code that uses the database instead of a file to keep track of status.

In [35]:
# open a new cursor and get all the urls that have no S3 key
cur = conn.cursor()
cur.execute("SELECT url FROM jsonurls WHERE s3key is NULL")
urls = cur.fetchall()

In [ ]:
# results come back as tuples, we want the first item in the tuple as the url
for t in urls:
    _url = t[0]